# **Librarys**

In [150]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# **Text Cleaning**

In [151]:
AR_STOPWORDS = set([
    'في', 'من', 'على', 'عن', 'إلى', 'أن', 'إن', 'كان', 'ما', 'لم', 'لن', 'قد', 'هذا', 'هذه', 'هناك',
    'هو', 'هي', 'هم', 'نحن', 'كما', 'أو', 'ثم', 'لكن', 'بل', 'أيضاً', 'حتى', 'مع', 'كل', 'أكثر', 'أقل',
    'لقد', 'ولا', 'له', 'منه', 'فيه', 'بين', 'بعد', 'عند', 'إلي', 'بدون', 'أحد', 'أي', 'شيء', 'لي',
    'به', 'فهو', 'فهي', 'كانت', 'ليس', 'لان', 'إذا', 'مثل', 'ضمن', 'أصبح', 'ذلك', 'والتي', 'والذي',
    'وهو', 'وهي', 'الذي', 'التي', 'الذين', 'اللذين', 'اللتي', 'بهذا', 'بها', 'بهم', 'بأن', 'فإن',
    'كما', 'إذ', 'حيث', 'أين', 'منذ', 'إما', 'أثناء', 'بينما', 'لأن', 'كأن', 'إلا', 'قد', 'حتي',
    'أو', 'و', 'ب', 'ل', 'ف', 'س', 'ك', 'ي', 'ت', 'ن', 'ا', 'إنه', 'أنها', 'وإن', 'أن', 'إن', 'مازال',
    'مايزال', 'مافتئ', 'ماانفك', 'ما برح', 'لايزال', 'ليس', 'ما', 'لاتزال', 'لاتزال', 'لن', 'لم', 'لا'
])

def clean_arabic_text(text):
    text = str(text).lower()
    text = re.sub(r'^\s*\d+\s*\.*\s*', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [w for w in words if w not in AR_STOPWORDS and len(w) > 1]
    return ' '.join(words)

# **Reading Data & Preparation**

In [152]:
data = pd.read_csv('/content/Untitled spreadsheet - cleaned_data - cleaned_data(1)(1).csv')
data

,symptoms_text,specialty
0,بحس إن قلبي بيخبط كأنه هيطلع من مكانه,قلب
1,نفسي بيتحبس فجأة ومش عارف ليه,قلب
2,كل لما أطلع سلم بحس إني هقع,قلب
3,وشي بيصفر لما دقات قلبي تزيد,قلب
4,بحس بخنقة في صدري من غير مجهود,قلب
...,...,...
2995,عندي ألم في ضهري بعد الوقوف الطويل ومأثرة على ...,علاج طبيعي
2996,بحس بشد جامد في العضلات ومش عارف أعمل إيه,علاج طبيعي
2997,رجلي ضعيفة بعد الكسور ومأثرة على حياتي,علاج طبيعي
2998,مش قادر أرفع دراعي لفوق بعد العملية ومش عارف أ...,علاج طبيعي


# **Data Spliting**

In [153]:
X = data['symptoms_text']
y = data['specialty']

counts = y.value_counts()
data = data[data['specialty'].isin(counts[counts >= 2].index)]

X = data['symptoms_text']
y = data['specialty']

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **MODEL PIPELINE**

In [155]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])

# **Tranning**

In [156]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                                 token_pattern='\\b\\w+\\b')),
                ('clf', LogisticRegression(max_iter=1000))])

# **predict**

In [159]:
y_pred = pipeline.predict(X_test)
y_pred

array(['أورام', 'الأعصاب', 'أورام', 'أسنان', 'سكر', 'الأعصاب',
       'مسالك بوليه', 'جلديه', 'عيون ورمد', 'سكر', 'باطنة', 'صدر', 'صدر',
       'أنف وأذن وحنجرة', 'مرض نفسي', 'نساء وتوليد', 'الأعصاب',
       'عيون ورمد', 'مرض نفسي', 'سكر', 'قلب', 'أسنان', 'نساء وتوليد',
       'جلديه', 'عظام', 'علاج طبيعي', 'مرض نفسي', 'عظام', 'أسنان', 'عظام',
       'الأعصاب', 'أنف وأذن وحنجرة', 'صدر', 'علاج طبيعي', 'أسنان', 'صدر',
       'جلديه', 'أسنان', 'أسنان', 'عظام', 'قلب', 'قلب', 'أسنان', 'صدر',
       'مسالك بوليه', 'مسالك بوليه', 'سكر', 'قلب', 'سكر',
       'أنف وأذن وحنجرة', 'الأعصاب', 'جلديه', 'أسنان', 'سكر', 'قلب',
       'علاج طبيعي', 'عظام', 'صدر', 'مرض نفسي', 'الأعصاب', 'صدر', 'صدر',
       'نساء وتوليد', 'أسنان', 'قلب', 'قلب', 'عظام', 'مسالك بوليه',
       'مسالك بوليه', 'عيون ورمد', 'أنف وأذن وحنجرة', 'مرض نفسي',
       'مرض نفسي', 'أورام', 'صدر', 'صدر', 'باطنة', 'مرض نفسي', 'باطنة',
       'سكر', 'أورام', 'أورام', 'الأعصاب', 'أورام', 'جلديه', 'صدر', 'سكر',
       'عظام', 'سكر', 'علاج

# **Evaluation & Report**

In [160]:
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

          أسنان       1.00      0.98      0.99        41
أنف وأذن وحنجرة       0.98      1.00      0.99        42
          أورام       1.00      1.00      1.00        38
        الأعصاب       0.92      0.97      0.95        37
          باطنة       0.97      1.00      0.99        34
          جلديه       1.00      1.00      1.00        39
            سكر       1.00      1.00      1.00        39
            صدر       0.88      0.95      0.91        37
           عظام       1.00      1.00      1.00        46
     علاج طبيعي       1.00      1.00      1.00        35
      عيون ورمد       1.00      0.96      0.98        51
            قلب       0.96      0.88      0.91        49
       مرض نفسي       1.00      1.00      1.00        39
    مسالك بوليه       1.00      1.00      1.00        34
    نساء وتوليد       0.95      0.95      0.95        39

       accuracy                           0.98       600
      macro avg       0.98   

In [161]:
def predict_specialty(text):
    clean = clean_arabic_text(text)
    return pipeline.predict([clean])[0]

print(predict_specialty("عندي ألم في الصدر وسعال متكرر"))

نساء وتوليد


# **Save Model**

In [162]:
with open('arabic_symptom_model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

print("✅ الموديل محفوظ باسم: arabic_symptom_model.pkl")

✅ الموديل محفوظ باسم: arabic_symptom_model.pkl
